In [2]:
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
from keras.models import model_from_json
from keras.optimizers import Adam
import json
from unet.losses import * 
warnings.filterwarnings('ignore')

img_size = 512
start_neurons = 32
k_size =3
LR = 1e-3
working_path = os.getcwd()+"/"
model_name = 'unet.hdf5'
print (working_path)

json_file = open(working_path + 'model.json', 'r')
json_load=json.load(json_file)
model = model_from_json(json_load)
model.compile(optimizer=Adam(lr=1e-3), loss=dice_coef_loss, metrics=[dice_coef])
model.load_weights(working_path + model_name)
model.summary()

/home/chen/u-net/2020-05-18-16-33/
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512, 512, 1)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 512, 512, 32)  320         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 512, 512, 32)  128         conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 512, 512, 32)  9248        batch_normalization_1[0][0]      
________________________________________________________

In [11]:
# summarize filter shapes
conv_l = []
for layer in model.layers:
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)
    


conv2d_1 (3, 3, 1, 32)
conv2d_2 (3, 3, 32, 32)
conv2d_3 (3, 3, 32, 64)
conv2d_4 (3, 3, 64, 64)
conv2d_5 (3, 3, 64, 128)
conv2d_6 (3, 3, 128, 128)
conv2d_7 (3, 3, 128, 256)
conv2d_8 (3, 3, 256, 256)
conv2d_9 (3, 3, 256, 512)
conv2d_10 (3, 3, 512, 512)
conv2d_transpose_1 (3, 3, 256, 512)
conv2d_11 (3, 3, 512, 256)
conv2d_12 (3, 3, 256, 256)
conv2d_transpose_2 (3, 3, 128, 256)
conv2d_13 (3, 3, 256, 128)
conv2d_14 (3, 3, 128, 128)
conv2d_transpose_3 (3, 3, 64, 128)
conv2d_15 (3, 3, 128, 64)
conv2d_16 (3, 3, 64, 64)
conv2d_transpose_4 (3, 3, 32, 64)
conv2d_17 (3, 3, 64, 32)
conv2d_18 (3, 3, 32, 32)
conv2d_19 (1, 1, 32, 1)
conv2d_1


In [14]:
model.get_layer('conv2d_8').Trainable=False
model.get_layer('conv2d_9').Trainable=False
model.get_layer('conv2d_10').Trainable=False
model.get_layer('conv2d_11').Trainable=False
model.get_layer('conv2d_12').Trainable=False
model.get_layer('conv2d_13').Trainable=False
model.compile(optimizer=Adam(lr=5e-5), loss=dice_coef_loss, metrics=[dice_coef])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512, 512, 1)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 512, 512, 32)  320         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 512, 512, 32)  128         conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 512, 512, 32)  9248        batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [22]:
from unet.make_512_dataset import gen_chunk, gen_aug_chunk, get_img_list
import glob

valid_case_no=-1
debug=1
batch_size =4

root_dir = "/home/chen/material/"
test_dir_path ="/home/chen/u-net/"
valid_path = root_dir+"CT_valid/"
train_path = root_dir+"CT/"
test_path = root_dir+"CT_test/"

train_case_list = glob.glob(train_path+"*/")
valid_case_list = glob.glob(valid_path+"*/")
if valid_case_no != -1: # use valid/train separation according to the dir.
    all_case_list = train_case_list + valid_case_list
    valid_case_list = [all_case_list.pop(valid_case_no)] # a list, not a str!!!
    train_case_list = all_case_list
if debug == 1:
    train_case_list = train_case_list[:3]
    valid_case_list = valid_case_list[:1]

print ("="*10," valid & train set ","="*10)
print ("valid case:",valid_case_list)
print ("train case:",train_case_list)
print ("="*10," valid & train set ","="*10)

raw_all_train,seg_all_train=get_img_list(train_case_list)
raw_all_valid,seg_all_valid=get_img_list(valid_case_list)

train_gen = gen_chunk(raw_all_train,seg_all_train,batch_size=batch_size)
train_aug_gen = gen_aug_chunk(train_gen)
valid_gen=gen_chunk(raw_all_valid,seg_all_valid)

raw_len = len(raw_all_train)
valid_raw_len = len(raw_all_valid)
print ("lengths: ",raw_len,"(train) ",valid_raw_len,"(valid)")


==========  valid & train set  ==========
valid case: ['/home/chen/material/CT_valid/CEA4_postope_CTA/']
train case: ['/home/chen/material/CT/CEA8_preope/', '/home/chen/material/CT/CEA13_preope_CTA/', '/home/chen/material/CT/mcs8/']
==========  valid & train set  ==========
lengths:  1122 (train)  609 (valid)


In [24]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,CSVLogger,Callback
EARLY_STOP_PATIENCE = 20
REDUCE_LR_FACTOR = 0.5
REDUCE_LR_PATIENCE=5
REDUCE_LR_MIN_LR = 1e-7
model_checkpoint = ModelCheckpoint(working_path+"fixed_layer.hdf5", 
                                   monitor='val_dice_coef',mode="max",
                                   verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor="val_dice_coef",mode="max",
                               patience=EARLY_STOP_PATIENCE,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_dice_coef",mode="max",
                              factor=REDUCE_LR_FACTOR,patience=REDUCE_LR_PATIENCE,
                              min_lr=REDUCE_LR_MIN_LR,verbose=1,epsilon=0.005)
csv_logger = CSVLogger(working_path+"fixed_log.log")

In [ ]:
epochs=5

history = model.fit_generator(
        train_aug_gen,
        steps_per_epoch = raw_len/batch_size,
        epochs=epochs,
        verbose=1,
        validation_data= valid_gen,
        validation_steps = valid_raw_len/batch_size,
        callbacks=[early_stopping,model_checkpoint,reduce_lr,csv_logger]
        )

Epoch 1/5
210/280 [=====================>........] - ETA: 39s - loss: 0.2242 - dice_coef: 0.7758